In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('dataset/development_dataset.csv')
test = pd.read_csv('dataset/leaderboard_dataset.csv')

In [3]:
train['VAR14'] = train['VAR14'].replace('.',1)
train['VAR21'] = train['VAR21'].replace({'Low','Medium','High'},(0,1,2))

In [4]:
train['VAR14']=train['VAR14'].astype(str).astype(int)
train['VAR21']=train['VAR21'].astype(str).astype(int)

In [5]:
columns = ['VAR18']
train = train.drop(columns, axis=1)
test = test.drop(columns,axis=1)

In [6]:
X = train.iloc[:,:-1]
Y = train.iloc[:,-1]

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [8]:
model = LGBMClassifier()

In [24]:
model = LGBMClassifier(boosting_type='gbdt',max_depth=10,objective ='softmax',learning_rate=0.01,sub_feature=0.5,
                       metric = 'multi_logloss' ,num_leaves=100, max_bin=1000,num_iterations=1000)

In [25]:
model.fit(X,Y)

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.01, max_bin=1000,
               max_depth=10, metric='multi_logloss', min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
               n_jobs=-1, num_iterations=1000, num_leaves=100,
               objective='softmax', random_state=None, reg_alpha=0.0,
               reg_lambda=0.0, silent=True, sub_feature=0.5, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [11]:
preds = model.predict(x_test)

In [12]:
output = pd.DataFrame(preds)

In [13]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, preds, labels=[0, 1, 2])

array([[1010, 1375,    2],
       [ 237, 3467,    0],
       [  72,  567,   70]], dtype=int64)

In [14]:
test['VAR14'] = test['VAR14'].replace('.',0)
test['VAR14']=test['VAR14'].astype(str).astype(int)

In [15]:
prediction = model.predict(test)

In [16]:
output = pd.DataFrame()
output['VAR1'] = test['VAR1']
output['VAR21']=prediction

In [17]:
output['VAR21'] = output['VAR21'].replace((0,1,2),{'Low','Medium','High'})

In [21]:
output.to_csv(r'Neil_hanna_IITKGP_1.csv',header=None, index=False)